
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

## Aggregating and summarizing data

Now let's look at some powerful functions we can use to aggregate and summarize data. In this notebook, we will continue to work with hihger-order functions; this time we will apply them to arrays containing numerical data. Also, we will work with additional functions in Spark SQL  that can be helpful when presenting data. 

In this notebook, you will: 
* Apply higher-order functions to numeric data
* Use a `PIVOT` command to create Pivot tables
* Use `ROLLUP` and `CUBE` modifiers to generate subtotals
* Use window functions to perform operations on a group of rows
* Use Databricks visualization tools to visualize and share data

Run the cell below to set up our classroom environment.

In [0]:
%run ../Includes/Classroom-Setup

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

res1: Boolean = false

res2: Boolean = false

## Higher-order functions and numerical data

Each of the higher-order functions we worked with in the last lesson can also be used with numerical data. In this lesson, we demonstrate how each of the functions in the previous lesson work with numeric data, as well as explore some powerful new higher-order functions.

Run the next two cells to create and describe the table we will be working with. You may recognize this table from a previous lesson. Recall that it contains data measuring environmental variability in a collection of data centers. The table `DeviceData` contains the `temps` and `co2Level` arrays we use to demonstrate higher-order functions. 

In [0]:
%sql
-- Crear la tabla 
DROP TABLE IF EXISTS DCDataRaw;
CREATE TABLE DCDataRaw
USING parquet                           
OPTIONS (
    PATH "/mnt/training/iot-devices/data-centers/2019-q2-q3"
    );
    
CREATE TABLE IF NOT EXISTS DeviceData3   
USING parquet                               
WITH ExplodeSource                        
AS
  (
  SELECT 
  dc_id,
  to_date(date) AS date,
  EXPLODE (source)
  FROM DCDataRaw
  )
SELECT 
  dc_id,
  key device_type,                         
  date,
  value.description,
  value.ip,
  value.temps,
  value.co2_level co2Level
  
FROM ExplodeSource;

In [0]:
%sql
-- Info de la tabla
DESCRIBE DeviceData3;

col_name,data_type,comment
dc_id,string,null
device_type,string,null
date,date,null
description,string,null
ip,string,null
temps,array,null
co2Level,array,null


### Preview data

Let's take a look a sample fo the data so that we con better understand the array values. 

In [0]:
%sql
-- Muestra aleatoriamente 1 fila
SELECT 
  temps, 
  co2Level
FROM DeviceData3
TABLESAMPLE (1 ROWS)

temps,co2Level
"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)","List(1196, 1360, 1125, 1206, 1342, 1198)"


### Filter

Filter operates on arrays containing numeric data just the same as those with text data. In this case, let's imagine that we want to collect all temperatures above a given threshold. Run the cell below to view the example. 

In [0]:
%sql
-- Busqueda -> filtrando 
SELECT 
  temps, 
  FILTER(temps, t -> t > 18) highTemps
FROM DeviceData3

temps,highTemps
"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)","List(19, 23)"
"List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12)","List(26, 19)"
"List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12)",List(19)
"List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22)","List(20, 20, 24, 19, 22)"
"List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3)",List()
"List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12)","List(21, 22)"
"List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23)","List(21, 20, 23, 25, 22, 28, 23, 28, 19, 26, 23)"
"List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8)",List()
"List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21)","List(19, 21)"
"List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5)",List()


### Exists
Exists operates on arrays containing numeric data just the same as those with text data. Let's say that we want to flag the records whose temperatures have exceeded a given value. Run the cell below to view the example. 

In [0]:
%sql
-- Busqueda -> exist
SELECT 
  temps, 
  EXISTS(temps, t -> t > 23) highTempsFlag
FROM DeviceData3

temps,highTempsFlag
"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)",false
"List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12)",true
"List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12)",false
"List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22)",true
"List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3)",false
"List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12)",false
"List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23)",true
"List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8)",false
"List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21)",false
"List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5)",false


### Transform

When using `TRANSFORM` with numeric data, we can apply any built-in function meant to work with a single value or we can name our own set of operations to be applied to each value in the array. This data includes temperature readings taken in Celsius. Each row contains an array of 12 temperature readings. We can use `TRANSFORM` to convert each element of each array to Fahrenheit. To convert from Celsius to Fahrenheit, multiply the temperature in Celsius by 9, divide by 5, and then add 32.

Let's dissect the code below to better understand the function: 

`TRANSFORM(temps, t -> ((t * 9) div 5) + 32 ) temps_F`

**`TRANSFORM`** : the name of the higher-order function <br>
**`temps`** : the name of our input array <br>
**`t`** : the name of the iterator variable. You choose this name and then use it in the lambda function. It iterates over the array, cycling each value into the function one at a time. <br>
**`->`** :  Indicates the start of the function <br>
**` ((t * 9) div 5) + 32`** : This is the function. For each value in the input array, the value is multipled by 9 and then divided by 5. Then, we add 32. This is the formula for converting from Celcius to Fahrenheit.
Recall that TRANSFORM takes an array, an iterator, and an anonymous function as input. In the code below, temps is the column that holds the array and t is the iterator that cycles through the list. The anonymous function ((t * 9) div 5) + 32 will be applied to each value in the list.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We use [the function](https://spark.apache.org/docs/latest/api/sql/#div) **`div`** to divide one expression by another without including decimal values. This is strictly for neatness in this example. For operations where precision counts, you would use the `/` operator, which performs floating point division. 

In [0]:
%sql
-- Busqueda -> transformar la temperatura 
SELECT 
  temps temps_C,
  TRANSFORM (temps, t -> ((t * 9) div 5) + 32 ) temps_F
FROM DeviceData3;


temps_C,temps_F
"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)","List(60, 55, 66, 51, 48, 73, 64, 55, 64, 62, 53, 53)"
"List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12)","List(78, 62, 66, 55, 48, 53, 50, 53, 33, 55, 60, 53)"
"List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12)","List(51, 55, 66, 46, 57, 60, 55, 57, 57, 48, 44, 53)"
"List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22)","List(68, 64, 68, 64, 51, 57, 62, 75, 62, 59, 66, 71)"
"List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3)","List(37, 33, 44, 46, 35, 25, 44, 33, 32, 46, 29, 37)"
"List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12)","List(51, 51, 60, 69, 46, 71, 55, 59, 50, 53, 57, 53)"
"List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23)","List(69, 68, 73, 77, 71, 82, 73, 82, 64, 66, 78, 73)"
"List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8)","List(48, 39, 59, 50, 41, 48, 46, 46, 33, 42, 42, 46)"
"List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21)","List(60, 57, 50, 53, 62, 50, 55, 51, 46, 66, 51, 69)"
"List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5)","List(51, 59, 51, 41, 55, 57, 51, 53, 48, 48, 51, 41)"



### Reduce 
`REDUCE` is more advanced than `TRANSFORM`; it takes two lambda functions. You can use it to reduce the elements of an array to a single value by merging the elements into a buffer, and applying a finishing function on the final buffer. 

We will use the reduce function to find an average value, by day, for our CO<sub>2</sub> readings. Take a closer look at the individual pieces of the `REDUCE` function by reviewing the list below. 

`REDUCE(co2_level, 0, (c, acc) -> c + acc, acc ->(acc div size(co2_level)))`


**`co2_level`** is the input array<br>
**`0`** is the starting point for the buffer. Remember, we have to hold a temporary buffer value each time a new value is added to from the array; we start at zero in this case to get an accurate sum of the values in the list. <br>
**`(c, acc)`** is the list of arguments we'll use for this function. It may be helpful to think of `acc` as the buffer value and `c` as the value that gets added to the buffer.<br>
**`c + acc`** is the buffer function. As the function iterates over the list, it holds the total (`acc`) and adds the next value in the list (`c`). <br>
**`acc div size(co2_level)`** is the finishing function. Once we have the sum of all numbers in the array, we divide by the number of elements to find the average. <br>

This view will be useful in subsequent exercises, so we’ll create a temporary view in this step so that we can access it later on. 

In [0]:
%sql
-- busqueda aplicando una funcion de reduccion, co2Level es el array, en contador inicia en 0, c,acc son los iteradores
CREATE OR REPLACE TEMPORARY VIEW Co2LevelsTemporary
AS
  SELECT
    dc_id, 
    device_type,
    co2Level,
    REDUCE(co2Level, 0, (c, acc) -> c + acc, acc ->(acc div size(co2Level))) as averageCo2Level
  
  FROM DeviceData3 
  SORT BY averageCo2Level DESC;

SELECT * FROM Co2LevelsTemporary

dc_id,device_type,co2Level,averageCo2Level
dc-103,sensor-istick,"List(1819, 1705, 1658, 1753, 1616, 1871)",1737
dc-103,sensor-ipad,"List(1617, 1607, 1835, 1783, 1726, 1568)",1689
dc-103,sensor-inest,"List(1595, 1684, 1682, 1631, 1688, 1754)",1672
dc-103,sensor-inest,"List(1633, 1651, 1753, 1913, 1526, 1552)",1671
dc-103,sensor-inest,"List(1633, 1708, 1550, 1649, 1829, 1593)",1660
dc-102,sensor-inest,"List(1850, 1610, 1589, 1720, 1581, 1598)",1658
dc-103,sensor-inest,"List(1698, 1705, 1682, 1626, 1705, 1535)",1658
dc-103,sensor-istick,"List(1523, 1631, 1835, 1642, 1526, 1781)",1656
dc-103,sensor-ipad,"List(1570, 1762, 1768, 1536, 1667, 1583)",1647
dc-103,sensor-inest,"List(1706, 1702, 1665, 1641, 1496, 1636)",1641


### Other higher-order functions
There are many built-in functions designed to work with array type data and well as other higher-order functions to explore. You can import [this notebook](https://docs.databricks.com/_static/notebooks/apache-spark-2.4-functions.html?_ga=2.12496948.1216795462.1586360468-278368669.1586265166) for a list of examples. 


### Pivot tables: Example 1
Pivot tables are supported in Spark SQL. A pivot table allows you to transform rows into columns and group by any data field. Let's take a closer look at our query. 

**`SELECT * FROM ()`**: The `SELECT` statement inside the parentheses in the input for this table. Note that it takes two columns from the view `Co2LevelsTemporary` <br>
**`PIVOT`**: The first argument in the clause is an aggregate function and the column to be aggregated. Then, we specify the pivot column in the `FOR` subclause. The `IN` operator contains the pivot column values. <br>

In [0]:
%sql
SELECT * FROM (
  SELECT device_type, averageCo2Level 
  FROM Co2LevelsTemporary
)

device_type,averageCo2Level
sensor-istick,1737
sensor-ipad,1689
sensor-inest,1672
sensor-inest,1671
sensor-inest,1660
sensor-inest,1658
sensor-inest,1658
sensor-istick,1656
sensor-ipad,1647
sensor-inest,1641


In [0]:
%sql
-- crear una tabla pivot
SELECT * FROM (
  SELECT device_type, averageCo2Level 
  FROM Co2LevelsTemporary
)
PIVOT (
  ROUND(AVG(averageCo2Level), 2) avg_co2 
  FOR device_type IN ('sensor-ipad', 'sensor-inest', 
    'sensor-istick', 'sensor-igauge')
  );

sensor-ipad,sensor-inest,sensor-istick,sensor-igauge
1245.98,1250.41,1244.86,1247.56


### Pivot Tables: Example 2

In this example, we again pull data from our larger table `DeviceData`. Within the subquery, we create the `month` column and use the `REDUCE` function to create the `averageCo2Level` column. 

In the pivot, we take the average of of the `averageCo2Level` values grouped by month. Notice that we rename the month columns from their number to the english abbreviations. 

Learn more about pivot tables in this [blog post](https://databricks.com/blog/2018/11/01/sql-pivot-converting-rows-to-columns.html).

In [0]:
%sql
-- crear una tabla pivot
SELECT
  *
FROM
  (
    SELECT
      month(date) month,
      REDUCE(co2Level, 0, (c, acc) -> c + acc, acc ->(acc div size(co2Level))) averageCo2Level
    FROM
      DeviceData3
  ) PIVOT (
    avg(averageCo2Level) avg FOR month IN (7 JUL, 8 AUG, 9 SEPT, 10 OCT, 11 NOV)
  )

JUL,AUG,SEPT,OCT,NOV
1242.8850806451612,1250.8649193548388,1245.1229166666667,1249.2983870967741,1247.7875


### Rollups

Rollups are operators used with the `GROUP BY` clause. They allow you to summarize data based on the columns passed to the `ROLLUP` operator. 

This results of this query include average CO<sub>2</sub> levels, grouped by `dc_id` and `device_type`. Rollups are creating subtotals for a specific group of data. The subtotals introduce new rows, and the new rows will contain `NULL` values for everything other than the calculated subtotal. 

We can alter this by using the `COALESCE()` function introduced in a previous lesson. 

In [0]:
%sql
-- agg para combinaciones específicas de valores en las columnas especificadas
-- resumir datos en función de las columnas pasadas al operador ROLLUP (crea resumenes)
-- COALESCE valor nulo en la columna retorna el valor en ""
SELECT 
  COALESCE(dc_id, "All data centers") AS dc_id,
  COALESCE(device_type, "All devices") AS device_type,
  ROUND(AVG(averageCo2Level))  AS avgCo2Level
FROM Co2LevelsTemporary
GROUP BY ROLLUP (dc_id, device_type)
ORDER BY dc_id, device_type;

dc_id,device_type,avgCo2Level
All data centers,All devices,1247.0
dc-101,All devices,1197.0
dc-101,sensor-igauge,1202.0
dc-101,sensor-inest,1197.0
dc-101,sensor-ipad,1194.0
dc-101,sensor-istick,1196.0
dc-102,All devices,1296.0
dc-102,sensor-igauge,1303.0
dc-102,sensor-inest,1294.0
dc-102,sensor-ipad,1293.0


In [0]:
%sql
SELECT 
  COALESCE(dc_id, "All data centers") AS dc_id,
  COALESCE(device_type, "All devices") AS device_type,
  ROUND(AVG(averageCo2Level))  AS avgCo2Level
FROM Co2LevelsTemporary
GROUP BY dc_id, device_type
ORDER BY dc_id, device_type;

dc_id,device_type,avgCo2Level
dc-101,sensor-igauge,1202.0
dc-101,sensor-inest,1197.0
dc-101,sensor-ipad,1194.0
dc-101,sensor-istick,1196.0
dc-102,sensor-igauge,1303.0
dc-102,sensor-inest,1294.0
dc-102,sensor-ipad,1293.0
dc-102,sensor-istick,1296.0
dc-103,sensor-igauge,1394.0
dc-103,sensor-inest,1408.0


### Cube
`CUBE` is also an operator used with the `GROUP BY` clause. Similar to `ROLLUP`, you can use `CUBE` to generate summary values for sub-elements grouped by column value. `CUBE` is different than `ROLLUP` in that it will also generate subtotals for all combinations of grouping columns specified in the `GROUP BY` clause. 

Notice that the output for the example below shows some of additional values generated in this query. Data from `"All data centers"` has been aggregated across device types for all centers. 

In [0]:
%sql
-- similar al rollup pero el cube tiene mas flexibilidad, ya que genera todas las combinaciones
SELECT 
  COALESCE(dc_id, "All data centers") AS dc_id,
  COALESCE(device_type, "All devices") AS device_type,
  ROUND(AVG(averageCo2Level))  AS avgCo2Level
FROM Co2LevelsTemporary
GROUP BY CUBE (dc_id, device_type)
ORDER BY dc_id, device_type;

dc_id,device_type,avgCo2Level
All data centers,All devices,1247.0
All data centers,sensor-igauge,1248.0
All data centers,sensor-inest,1250.0
All data centers,sensor-ipad,1246.0
All data centers,sensor-istick,1245.0
dc-101,All devices,1197.0
dc-101,sensor-igauge,1202.0
dc-101,sensor-inest,1197.0
dc-101,sensor-ipad,1194.0
dc-101,sensor-istick,1196.0


In [0]:
%run ../Includes/Classroom-Cleanup


&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>